## Initial database setup.


### Create an encrypted file with all the database passwords using [Fernet](https://cryptography.io/en/latest/fernet/).

1. Import relevant libs. Use fernet from cryptography lib.

In [1]:
from cryptography.fernet import Fernet
import json

from pymongo import MongoClient
import csv

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

/home/james/anaconda3/envs/anacenv/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/james/anaconda3/envs/anacenv/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


2. Create dictionary with passwords and master password (Delete this cell after use!).

In [15]:
password_dict = {"Obsmanager": "obspassword",
                 "Sensorwriter": "senpassword",
                 "Magnetwriter": "magpassword",
                 "Datreader": "readpassword"}

# Keep this safe!
key = b'bZ54iqDolhe8n-UBJ8wCwOgN61a9_Ef_TUX7HUB5LIE='#Fernet.generate_key()
print("The key is:", key)

The key is: b'bZ54iqDolhe8n-UBJ8wCwOgN61a9_Ef_TUX7HUB5LIE='


3. Encrypt passwords.

In [16]:
f = Fernet(key)
token = f.encrypt(bytes(json.dumps(password_dict), "utf-8"))
with open("access.crypt", 'wb') as cr_file:
    cr_file.write(token)

del password_dict, f, token

4. Try reloading passwords.

In [17]:
f = Fernet(key)
with open("access.crypt", 'rb') as cr_file:
    password_dict = json.loads(f.decrypt(cr_file.read()).decode("utf-8"))
    
print(password_dict)

{'Obsmanager': 'obspassword', 'Sensorwriter': 'senpassword', 'Magnetwriter': 'magpassword', 'Datreader': 'readpassword'}


### Add a sensor to the database.
1. Connect to the database.

In [8]:
# Get the connection and select the "sensorDat" collection in the "mag_db" database.
client = MongoClient("localhost",
                      username="Obsmanager",
                      password=password_dict["Obsmanager"],
                      authSource="mag_db")["mag_db"]["observatories"]

2. Add the new observatories.

In [10]:
with open("observatories.txt", 'r') as obs_file:
    for obs in obs_file:
        obs_dat = json.loads(obs)
        print(obs_dat)
        client.insert_one(obs_dat)

{'_id': 'frg_single_fgm3', 'Location': 'Frongoch Farm', 'latlon': None, 'coord_sys': 'H'}


### Write raw sensor data to the database.

1. Connect to the database.

In [12]:
# Get a connection to the mongo database.
# Get the connection and select the "sensorDat" collection in the "mag_db" database.
client = MongoClient("localhost",
                      username="Sensorwriter",
                      password=password_dict["Sensorwriter"],
                      authSource="mag_db")["mag_db"]["sensorDat"]

2. Load sensor data and write to the database.

In [14]:
dat_fp = "../mag_data_copy/"
years = ["2011", "2012", "2013", "2014", "2015", "2016", "2017", "2018"]
# Batch size to load data into mongodb. Set to load 10000 items at a time.
batch_sz = 10000


for year in years:
    for chunk in pd.read_csv(dat_fp+year+"_mag_dat.csv",
                             dtype={"time": np.object_,
                                    "reading": np.float64,
                                    "temperature": np.float64},
                             chunksize=batch_sz):
        chunk["time"] = pd.to_datetime(chunk["time"],format="%Y-%m-%d %H:%M:%S")
        chunk = chunk.loc[chunk["time"].notnull()]
        chunk["sensor_id"] = "frg_single_fgm3"
        client.insert_many(chunk.T.to_dict().values())